In [65]:
import tensorflow as tf

In [66]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
from tensorflow.python.framework import ops
ops.reset_default_graph()
%matplotlib inline

In [67]:
data = pd.read_csv('Untitled Document 1.csv',names=['l1','l2','l3','l4','l5','l6','l7'])

In [68]:
p = np.asarray([1,0])
a = np.asarray([0,1])
data['l7'] = data['l7'].map({1:p,2:a})

In [69]:
data['l1'] = data['l1']/103
data['l2'] = data['l2']/138
data['l3'] = data['l3']/155
data['l4'] = data['l4']/182
data['l5'] = data['l5']/297
data['l6'] = data['l6']/20



In [70]:
data[:][:5]

,l1,l2,l3,l4,l5,l6,l7
0,0.825243,0.666667,0.290323,0.148352,0.104377,0.0,"[1, 0]"
1,0.825243,0.463768,0.380645,0.175824,0.077441,0.0,"[0, 1]"
2,0.834951,0.391304,0.212903,0.087912,0.181818,0.0,"[0, 1]"
3,0.883495,0.565217,0.219355,0.131868,0.121212,0.0,"[0, 1]"
4,0.844660,0.507246,0.077419,0.153846,0.033670,0.0,"[0, 1]"


In [71]:
from sklearn.model_selection import train_test_split

In [72]:
X = data[['l1','l2','l3','l4','l5','l6']]
y = data['l7']

In [73]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [74]:
n_input = 6
n_hidden_1 = 200
n_classes = 2


In [75]:
x=tf.placeholder(tf.float32,shape=[None,6])
y_=tf.placeholder(tf.float32,shape=[None, 2])

In [76]:
def gaussian_function(input_layer):
    initial = math.exp(-2*math.pow(input_layer, 2))
    return initial

np_gaussian_function = np.vectorize(gaussian_function)

def d_gaussian_function(input_layer):
    initial = -4 * input_layer * math.exp(-2*math.pow(input_layer, 2))
    return initial

np_d_gaussian_function = np.vectorize(d_gaussian_function)

np_d_gaussian_function_32 = lambda input_layer: np_d_gaussian_function(input_layer).astype(np.float32)

def tf_d_gaussian_function(input_layer, name=None):
    with ops.name_scope(name, "d_gaussian_function", [input_layer]) as name:
        y = tf.py_func(np_d_gaussian_function_32, [input_layer],[tf.float32], name=name, stateful=False)
    return y[0]

def py_func(func, inp, Tout, stateful=True, name=None, grad=None):
    rnd_name = 'PyFunGrad' + str(np.random.randint(0, 1E+8))

    tf.RegisterGradient(rnd_name)(grad)
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)

def gaussian_function_grad(op, grad):
    input_variable = op.inputs[0]
    n_gr = tf_d_gaussian_function(input_variable)
    return grad * n_gr

np_gaussian_function_32 = lambda input_layer: np_gaussian_function(input_layer).astype(np.float32)

def tf_gaussian_function(input_layer, name=None):
    with ops.name_scope(name, "gaussian_function", [input_layer]) as name:
        y = py_func(np_gaussian_function_32, [input_layer], [tf.float32], name=name, grad=gaussian_function_grad)
    return y[0]
# end of defining activation function


In [77]:
def rbf_network(input_layer, weights):
    layer1 = tf.matmul(tf_gaussian_function(input_layer), weights['h1'])
    output = tf.matmul(tf_gaussian_function(layer1), weights['output'])
    return output

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=0.1)),
    
    'output': tf.Variable(tf.random_normal([n_hidden_1, n_classes], stddev=0.1))
}

In [78]:
pred = rbf_network(x, weights)

In [79]:
loss = tf.reduce_sum( tf.square( y_ - pred ) )

In [98]:
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [103]:
sess = tf.InteractiveSession()
init = tf.initialize_all_variables()
sess.run(init)
epoch=2000

/home/pulkitsharma97/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [104]:
for step in range(epoch):
   _, c=sess.run([train_step,loss], feed_dict={x: X_train, y_:[t for t in y_train.as_matrix()]})
   if step%500==0 :
       print(c)

/home/pulkitsharma97/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


163.81178
104.78495
97.674614
92.79536


In [105]:
print(sess.run(accuracy,feed_dict={x: X_test, y_:[t for t in y_test.as_matrix()]})) 

0.7105263


/home/pulkitsharma97/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
